# Problem Set #7
## Name: Weiwei Zheng

In [1]:
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
auto = pd.read_csv('Auto.csv', sep=',', na_values = "?")
auto.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      5
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [3]:
auto.dropna(inplace = True)
auto = auto.reset_index()
auto.head()

,index,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [4]:
y = (auto['mpg'] >= auto['mpg'].median()).astype(int)
X = auto[['cylinders', 'displacement', 'horsepower', 'weight', \
         'acceleration', 'year', 'origin']]

***a) logistic regression***

In [5]:
X.loc[:,'constant'] = 1
kf_log = KFold(n_splits = 4, shuffle = True, random_state = 15)
kf_log.get_n_splits(X)
error_df_log = pd.DataFrame({"error": np.zeros(0), "y_test": np.zeros(0), \
                              "y_pred": np.zeros(0)})

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
Xvals = X.values
yvals = y.values

In [7]:
MSE_log = 0
    
for train_index, test_index in kf_log.split(Xvals):
    X_train, X_test = Xvals[train_index], Xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    error_df_k = pd.DataFrame({"error": np.zeros(len(test_index)), \
                            "y_test": y_test, "y_pred": y_pred})
    error_df_k['error'] = (error_df_k["y_test"] != error_df_k["y_pred"]).astype(int)
    error_df_log = pd.concat([error_df_log, error_df_k])    
    MSE_log += error_df_k["error"].mean()

In [8]:
print(classification_report(error_df_log["y_test"], error_df_log["y_pred"]))

             precision    recall  f1-score   support

        0.0       0.91      0.88      0.89       196
        1.0       0.88      0.91      0.89       196

avg / total       0.89      0.89      0.89       392



In [9]:
print('test estimate MSE_log=', MSE_log/4)
print('error rate for mpg_high = 1 is', error_df_log[error_df_log["y_pred"] \
                                                     == 1]["error"].mean())
print('error rate for mpg_high = 0 is', error_df_log[error_df_log["y_pred"] \
                                                     == 0]["error"].mean())

test estimate MSE_log= 0.10714285714285715
error rate for mpg_high = 1 is 0.1188118811881188
error rate for mpg_high = 0 is 0.09473684210526316


***b) random forest ***

In [10]:
tree_random = RandomForestClassifier(n_estimators = 20, max_features = 2, \
                                     bootstrap = True, oob_score = True, 
                                     random_state = 25)
X_sub = auto[['cylinders', 'displacement', 'horsepower', 'weight', \
         'acceleration', 'year', 'origin']]
tree_random.fit(X_sub, y)
y_pred = pd.DataFrame(tree_random.oob_decision_function_.T[1], columns = ['y_pred'])
y_pred['y_pred'] = (y_pred['y_pred'] >= .5).astype(int)
mse_tree = mean_squared_error(y, y_pred)

In [11]:
print(classification_report(y, y_pred, digits = 3))

             precision    recall  f1-score   support

          0      0.942     0.913     0.927       196
          1      0.916     0.944     0.930       196

avg / total      0.929     0.929     0.929       392



In [12]:
print('test estimate MSE_tree=', mse_tree)
print('error rate for mpg_high = 1 is 0.084')
print('error rate for mpg_high = 0 is 0.058')

test estimate MSE_tree= 0.0714285714286
error rate for mpg_high = 1 is 0.084
error rate for mpg_high = 0 is 0.058


***c) support vector machine ***

In [18]:
kf_svm = KFold(n_splits = 4, shuffle = True, random_state = 15)
kf_svm.get_n_splits(X_sub)
error_df_svm = pd.DataFrame({"error": np.zeros(0), "y_test": np.zeros(0), \
                              "y_pred": np.zeros(0)})

In [19]:
MSE_svm = 0
    
for train_index, test_index in kf_svm.split(X_sub):
    X_train, X_test = X_sub.values[train_index], X_sub.values[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]    
    svm_rbf = svm.SVC(C=1, gamma = 0.2)
    svm_rbf.fit(X_train, y_train)
    y_pred = svm_rbf.predict(X_test)
    print('\n',classification_report(y_test, y_pred))   
    error_df_k = pd.DataFrame({"error": np.zeros(len(test_index)), \
                               "y_test": y_test, "y_pred": y_pred})
    error_df_k['error'] = (error_df_k["y_test"] != error_df_k["y_pred"]).astype(int)
    error_df_svm = pd.concat([error_df_svm, error_df_k])    
    MSE_svm += error_df_k["error"].mean()


              precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98


              precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98


              precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33        98


              precision    recall  f1-score   support

          0       0.53      1.00      0.69        49
          1       1.00      0.10      0.19        49

avg / total       0.76      0.55      0.44        98



C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
error_df_svm.groupby(['error', 'y_pred']).count()

y_test
error y_pred        
0.0   0.0        143
      1.0         50
1.0   0.0        146
      1.0         53

In [21]:
print(classification_report(error_df_svm["y_test"], error_df_svm["y_pred"], digits = 3))

             precision    recall  f1-score   support

        0.0      0.495     0.730     0.590       196
        1.0      0.485     0.255     0.334       196

avg / total      0.490     0.492     0.462       392



In [22]:
print('test estimate MSE_log=', MSE_svm/4)
print('error rate for mpg_high = 1 is', error_df_svm[error_df_svm["y_pred"] \
                                                     == 1]["error"].mean())
print('error rate for mpg_high = 0 is', error_df_svm[error_df_svm["y_pred"] \
                                                     == 0]["error"].mean())

test estimate MSE_log= 0.5076530612244898
error rate for mpg_high = 1 is 0.5145631067961165
error rate for mpg_high = 0 is 0.5051903114186851


***d) model comparision ***

Overall, random forest should be the best approach of the three to predict mpg_high, for it has the lowest MSE, and highes precision for both categories. 